<a href="https://colab.research.google.com/github/SheikhMudassarHanif/NLP/blob/main/CD_ENCLSTML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
#run this
excel_file_path = '/content/fulldatafullcleaned.csv'
column_names = pd.read_csv(excel_file_path, nrows=0).columns.tolist()
df = pd.read_csv(excel_file_path,usecols=[column_names[1],column_names[2]])
df.dropna(inplace=True)
df.head()

,SENTENCES,MEANING
0,how can i communicate with my parents,میں اپنے والدین سے کیسے بات کروں
1,how can i make friends,میں دوست کیسے بنائوں
2,why do i get so sad,میں اتنا اداس کیوں ہوں
3,if youve asked yourself such questions youre n...,اگر آپ نے اپنے آپ سے ایسے سوالات کیے ہیں تو آپ...
4,depending on where youve turned for guidance y...,اس بات پر منحصر ہے کہ آپ رہنمائی کے لیے کہاں گ...


In [ ]:
df = df.drop_duplicates()


In [ ]:

# Assuming df is your DataFrame containing the SENTENCES and MEANING columns
df = df.groupby('SENTENCES ').agg(lambda x: x.value_counts().index[0]).reset_index()


In [ ]:
df.describe()

,SENTENCES,MEANING
count,21650,21650
unique,21650,21107
top,_x_ although this chapter discusses those who ...,(گوگل کے ذریعہ ترجمہ شدہ) (اصل)
freq,1,35


In [ ]:

from nltk.tokenize import word_tokenize
from collections import Counter

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.tokenize import word_tokenize
from collections import Counter
import pandas as pd

from LughaatNLP import LughaatNLP
tokens_words = LughaatNLP()

# tokens = tokens_words.urdu_tokenize(text)
# Let's assume df is your DataFrame containing the sentences and meanings.
# Make sure to clean your DataFrame column names to avoid whitespace issues.

# Create Counters for both English and Urdu
english_word_counts = Counter()
urdu_word_counts = Counter()

# Define the threshold for word frequency
threshold = 3

# Process the 'SENTENCES' and 'MEANING' columns
for index, row in df.iterrows():
    # Tokenize and update English word counts
    english_text = row['SENTENCES ']  # Assuming this column contains English sentences
    english_tokens = word_tokenize(english_text)
    english_word_counts.update(english_tokens)

    # Tokenize and update Urdu word counts
    urdu_text = row['MEANING']
      # Assuming this column contains Urdu meanings
    urdu_tokens=tokens_words.urdu_tokenize(urdu_text)
    # urdu_tokens = word_tokenize(urdu_text)
    urdu_word_counts.update(urdu_tokens)

# Create the complete vocabulary for English and Urdu
english_vocabulary = set(english_word_counts.keys())
urdu_vocabulary = set(urdu_word_counts.keys())

# Get sizes of the vocabularies
english_vocabulary_size = len(english_vocabulary)
urdu_vocabulary_size = len(urdu_vocabulary)
print("English Vocabulary size:", english_vocabulary_size)
print("Urdu Vocabulary size:", urdu_vocabulary_size)

# Apply the threshold for both vocabularies
reduced_english_vocabulary = {word for word, count in english_word_counts.items() if count >= threshold}
reduced_urdu_vocabulary = {word for word, count in urdu_word_counts.items() if count >= threshold}

# Get reduced vocabulary sizes
reduced_english_vocabulary_size = len(reduced_english_vocabulary)
reduced_urdu_vocabulary_size = len(reduced_urdu_vocabulary)
print("Reduced English Vocabulary size:", reduced_english_vocabulary_size)
print("Reduced Urdu Vocabulary size:", reduced_urdu_vocabulary_size)

English Vocabulary size: 16211
Urdu Vocabulary size: 13893
Reduced English Vocabulary size: 6519
Reduced Urdu Vocabulary size: 6194


In [ ]:
# reduced_english_vocabulary

In [ ]:
word_to_index_eng={word: index for index, word in enumerate(reduced_english_vocabulary)}
word_to_index_urd={word: index for index, word in enumerate(reduced_urdu_vocabulary)}



In [ ]:
from keras.preprocessing.sequence import pad_sequences

def encode_tokens(sequences, word_to_index, max_length):
    encoded_sequences = []
    for tokens in sequences:
        encoded = [word_to_index.get(token, 0) for token in tokens]  # 0 for unknown words
        padded_encoded = pad_sequences([encoded], maxlen=max_length, padding='post', truncating='post')[0]
        encoded_sequences.append(padded_encoded)
    return np.array(encoded_sequences)

In [ ]:
def decode_tokens(encoded_sequence, index_to_word):
    # Convert indices to words, ignoring padding (index 0) if present
    decoded_tokens = [index_to_word.get(index, '<UNK>') for index in encoded_sequence if index != 0]

    return decoded_tokens


In [ ]:
# Ensure that both lists are created from the same dataframe or aligned data source
X_ecn = encode_tokens(df['SENTENCES '].tolist(), word_to_index_eng, max_length=10)
y_enc = encode_tokens(df['MEANING'].tolist(), word_to_index_urd, max_length=10)
X = np.array(X_ecn)  # Shape: (num_samples, max_length)
y = np.array(y_enc)
X.shape,y.shape


((21650, 10), (21650, 10))

In [ ]:
# Split the data into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)


# Define the combined vocabulary size (English + Urdu + padding token)
vocab_size = len(word_to_index_eng) + len(word_to_index_urd) + 1  # +1 for padding token

# No need to reshape y_train and y_val
# They should already be in shape (num_samples, max_length)

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=100, input_length=10),  # Combined vocab size for input
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),  # 1st Bidirectional LSTM
    tf.keras.layers.Dropout(0.5),  # Dropout layer
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),  # 2nd Bidirectional LSTM
    tf.keras.layers.Dropout(0.5),  # Dropout layer
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(vocab_size, activation='softmax'))  # TimeDistributed output layer
])

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print model summary
model.summary()

# Set up the callbacks
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath='LASTESTENCLSTM_MODEL_RV.keras', save_best_only=True, monitor='val_loss', verbose=1)
]

# Train the model
# model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=80, batch_size=32, callbacks=callbacks)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Interputing the model as it was stuck in a local maxmima

In [ ]:
model42ep=tf.keras.models.load_model('/content/LASTESTENCLSTM_MODEL_RV2Iep4.keras')

In [ ]:
model42ep.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print model summary
model42ep.summary()

# Set up the callbacks
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath='LASTESTENCLSTM_MODEL_RVep42.keras', save_best_only=True, monitor='val_loss', verbose=1)
]

# Train the model
histroy=model42ep.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=80, batch_size=32, callbacks=callbacks)

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)              │ (None, 10, 100)             │       1,271,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_14 (Bidirectional)     │ (None, 10, 256)             │         234,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 10, 256)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_15 (Bidirectional)     │ (None, 10, 256)             │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 10, 256)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_7 (TimeDistributed) │ (None, 10, 12714)           │       3,267,498 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,167,634 (19.71 MB)

 Trainable params: 5,167,634 (19.71 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/80
609/609 ━━━━━━━━━━━━━━━━━━━━ 0s 504ms/step - accuracy: 0.2982 - loss: 4.8991
Epoch 1: val_loss improved from inf to 2.22032, saving model to LASTESTENCLSTM_MODEL_RVep42.keras
609/609 ━━━━━━━━━━━━━━━━━━━━ 327s 525ms/step - accuracy: 0.2982 - loss: 4.8963 - val_accuracy: 0.3861 - val_loss: 2.2203
Epoch 2/80
609/609 ━━━━━━━━━━━━━━━━━━━━ 0s 490ms/step - accuracy: 0.3766 - loss: 2.2583
Epoch 2: val_loss improved from 2.22032 to 2.12994, saving model to LASTESTENCLSTM_MODEL_RVep42.keras
609/609 ━━━━━━━━━━━━━━━━━━━━ 313s 510ms/step - accuracy: 0.3766 - loss: 2.2582 - val_accuracy: 0.4109 - val_loss: 2.1299
Epoch 3/80
609/609 ━━━━━━━━━━━━━━━━━━━━ 0s 467ms/step - accuracy: 0.4000 - loss: 2.1702
Epoch 3: val_loss improved from 2.12994 to 2.07022, saving model to LASTESTENCLSTM_MODEL_RVep42.keras
609/609 ━━━━━━━━━━━━━━━━━━━━ 295s 484ms/step - accuracy: 0.4000 - loss: 2.1702 - val_accuracy: 0.4284 - val_loss: 2.0702
Epoch 4/80
609/609 ━━━━━━━━━━━━━━━━━━━━ 0s 447ms/step - accuracy: 0.419

KeyboardInterrupt: 

In [ ]:
model3I31ep=tf.keras.models.load_model('/content/LASTESTENCLSTM_MODEL_RVep42.keras')

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Load the saved model
model3I31ep=tf.keras.models.load_model('/content/LASTESTENCLSTM_MODEL_RVep42.keras')

# Re-compile the model with RMSprop optimizer
optimizer = RMSprop(learning_rate=0.001)  # You can adjust the learning rate as needed
model3I31ep.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Set up the callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True, verbose=1)

callbacks = [reduce_lr, early_stopping]

# Now you can continue training
history_model3I31ep = model3I31ep.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=32, callbacks=callbacks)

Epoch 1/5
 34/609 ━━━━━━━━━━━━━━━━━━━━ 4:35 479ms/step - accuracy: 0.5214 - loss: 1.7117

In [32]:
model3I31ep.save('LastestLSTMModel52Acc.keras')

In [36]:
sentence='Hello how are you '
encoded=encode_tokens([sentence],word_to_index_eng,max_length=10)
predictions=model.predict(encoded)
# print(predictions)
encode_predictions=encode_tokens(predictions,word_to_index_urd,max_length=10)
print(decode_tokens(encode_predictions[0],word_to_index_urd))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


TypeError: unhashable type: 'numpy.ndarray'

In [33]:
# import matplotlib.pyplot as plt
# plt.plot(history_model3I31ep.history['loss'], label='Training Loss')
# plt.plot(history_model3I31ep.history['val_loss'], label='Validation Loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

In [38]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences


def decode_prediction(predicted_indices, index_to_word):
    # Convert predicted indices back to words
    return ' '.join([index_to_word.get(index, '') for index in predicted_indices if index > 0])  # Exclude padding (0)

# Example test sentence
test_sentence = "hello how are you"

# Encode the test sentence
X_test = encode_tokens(test_sentence, word_to_index_eng, max_length=10)

# Make a prediction
predicted_indices = model.predict(X_test)

# Assuming predicted_indices is of shape (1, max_length) (for one input)
predicted_indices = np.argmax(predicted_indices, axis=-1)  # Take the index with the highest score

# Create a reverse mapping for Urdu indexes
index_to_word_urd = {index: word for word, index in word_to_index_urd.items()}

# Decode the prediction
predicted_meaning = decode_prediction(predicted_indices[0], index_to_word_urd)

print(f"Predicted meaning: {predicted_meaning}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 681ms/step
Predicted meaning: سمجھے         


In [43]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences


def decode_prediction(predicted_indices, index_to_word):
    # Convert predicted indices back to words
    return ' '.join([index_to_word.get(index, '') for index in predicted_indices if index > 0])  # Exclude padding (0)

# Example test sentence
test_sentence = "."

# Encode the test sentence
X_test = encode_tokens(test_sentence, word_to_index_eng, max_length=10)

# Make a prediction
predicted_indices = model.predict(X_test)

# Assuming predicted_indices is of shape (1, max_length) (for one input)
predicted_indices = np.argmax(predicted_indices, axis=-1)  # Take the index with the highest score

# Create a reverse mapping for Urdu indexes
index_to_word_urd = {index: word for word, index in word_to_index_urd.items()}

# Decode the prediction
predicted_meaning = decode_prediction(predicted_indices[0], index_to_word_urd)

print(f"Predicted meaning: {predicted_meaning}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Predicted meaning: سمجھے         
